In [1]:
from pandas import read_csv, Series
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
from pickle import load
from glove import Glove
import adagram
from gensim.models.wrappers import FastText, Wordrank
from embed_utils import Word2VecF, Swivel, cosine_sim, get_adagram_sense_prob, wv
from utils.string_utils import morph_parse, make_tokens

In [2]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.dummy import DummyClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import (accuracy_score, brier_score_loss, precision_score, recall_score, f1_score)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [3]:
import matplotlib.pyplot as plt
import seaborn

In [23]:
def get_feature_vec(tokens, num_features, model):
    featureVec = np.zeros(shape=(1, num_features), dtype='float32')
    nwords = 0
    for word in tokens:
        if model == 'word2vec':
            featureVec = np.add(featureVec, word2vec[word])
        elif model == 'wang2vec':
            featureVec = np.add(featureVec, wang2vec[word])
        elif model == 'glove':
            featureVec = np.add(featureVec, wv(glove, word))
        elif model == 'word2vecf':
            featureVec = np.add(featureVec, w2vf.word2vec(word))
        elif model == 'adagram':
            featureVec = np.add(featureVec, ada_model.sense_vector(word, get_adagram_sense_prob(ada_model, word)))
        elif model == 'fasttext':
            featureVec = np.add(featureVec, ft[word])
        elif model == 'bow':
            featureVec = np.add(featureVec, bow[word])
        elif model == 'swivel':
            featureVec = np.add(featureVec, swivel.lookup(word))
        nwords = nwords + 1
    try:
        featureVec = np.divide(featureVec, nwords)
    except FloatingPointError:
         featureVec = np.zeros(shape=(1,num_features), dtype='float32')
    return featureVec

In [24]:
old_err_state = np.seterr(all='raise')

def vectorize_message(message1, message2, model, num_features, vocab):
    tokens1 = make_tokens(message1.lower(), vocab)
    tokens2 = make_tokens(message2.lower(), vocab)
    fv1 = get_feature_vec(tokens1, num_features, model)
    fv2 = get_feature_vec(tokens2, num_features, model)
    return np.hstack((fv1, fv2))

In [25]:
df = read_csv('anno2ch/annotated.csv', encoding='cp1251')
df.comment = df.comment.apply(morph_parse)
df.reference = df.reference.apply(morph_parse)

In [26]:
df_unl = read_csv('anno2ch/annotated.csv', encoding='cp1251')

In [27]:
Y = df['labels'].values

Загрузка Word2Vec-модели

In [28]:
word2vec = Word2Vec.load('models/word2vec/all_lem_100')
word2vec_vocab = word2vec.wv.vocab

Загрузка Glove-модели

In [10]:
with open('models/glove/all_lem_100', 'rb') as fp:
    glove = load(fp)
glove_vocab = glove.dictionary

Загрузка Wang2Vec-модели

In [11]:
wang2vec = KeyedVectors.load_word2vec_format('models/wang2vec/all_lem_100_cwindow', binary=True)
wang2vec_vocab = wang2vec.vocab

Загрузка Word2Vec-f-модели

In [12]:
from os import path

w2vf = Word2VecF.load(path.join('models/word2vecf', 'vecs.npy'), path.join('models/word2vecf', 'vecs.vocab'))
w2vf_vocab = w2vf._vocab

Загрузка Adagram-модели

In [13]:
ada_model = adagram.VectorModel.load('models/adagram/all_lem_100.pkl')
adagram_vocab = ada_model.dictionary.word2id.keys()

Загрузка BOW-модели

In [14]:
from pickle import load

with open('models/tf_idf/all_lem', 'rb') as fp:
    bow = load(fp)

Загрузка Swivel-модели

In [15]:
swivel = Swivel('models/swivel/vocab_100.txt', 'models/swivel/vecs_100.bin')
swivel_vocab = swivel.vocab

Загрузка Fasttext-модели

In [16]:
ft = FastText.load_word2vec_format('models/fasttext/all_100_skipgram.vec')
ft_vocab = ft.vocab

Загрузка Wordrank-модели

In [17]:
#wr = Wordrank()
#mod = Wordrank.load_wordrank_model('/media/defeater/d5233b78-d9c7-40e7-a454-8b108bcc4b8a/defeater/NLP/wordrank/meta/cooccurrence')
#wr = wr.train('/media/defeater/d5233b78-d9c7-40e7-a454-8b108bcc4b8a/defeater/NLP/wordrank', corpus_file='corpus.txt', out_path='/media/defeater/d5233b78-d9c7-40e7-a454-8b108bcc4b8a/defeater/NLP/wordrank')

Получение датасетов

In [29]:
word2vec_vectors = np.zeros(shape=(len(df),200), dtype='float32')

for i, m in df.iterrows():
    word2vec_vectors[i] = vectorize_message(m['comment'], m['reference'], 'word2vec', 100, word2vec_vocab)

In [30]:
glove_vectors = np.zeros(shape=(len(df),200), dtype='float32')

for i, m in df.iterrows():
    glove_vectors[i] = vectorize_message(m['comment'], m['reference'], 'glove', 100, glove_vocab)

In [31]:
wang2vec_vectors = np.zeros(shape=(len(df),200), dtype='float32')

for i, m in df.iterrows():
    wang2vec_vectors[i] = vectorize_message(m['comment'], m['reference'], 'wang2vec', 100, wang2vec_vocab)

In [32]:
adagram_vectors = np.zeros(shape=(len(df),200), dtype='float32')

for i, m in df.iterrows():
    adagram_vectors[i] = vectorize_message(m['comment'],m['reference'], 'adagram', 100, adagram_vocab)

In [34]:
w2vf_vectors = np.zeros(shape=(len(df),200), dtype='float32')

for i, m in df_unl.iterrows():
    w2vf_vectors[i] = vectorize_message(m['comment'],m['reference'], 'word2vecf', 100, w2vf_vocab)

In [35]:
ft_vectors = np.zeros(shape=(len(df),200), dtype='float32')

for i, m in df.iterrows():
    ft_vectors[i] = vectorize_message(m['comment'],m['reference'], 'fasttext', 100, ft_vocab)

In [36]:
bow_vectors = np.zeros(shape=(len(df),2), dtype='float32')

for i, m in df.iterrows():
    bow_vectors[i] = vectorize_message(m['comment'], m['reference'], 'bow', 1, bow)

In [37]:
swivel_vectors = np.zeros(shape=(len(df), 200), dtype='float32')

for i, m in df.iterrows():
    swivel_vectors[i] = vectorize_message(m['comment'], m['reference'], 'swivel', 100, swivel_vocab)

Сравнение

In [38]:
def set_plt_params():
    title_font = {'size':'30', 'color':'black', 'weight':'normal',
                  'verticalalignment':'bottom'} 
    axis_font = {'size':'30'}

    plt.figure(figsize=(40, 20))
    plt.xlabel('Training examples', **axis_font)
    plt.ylabel('Score',  **axis_font)
    plt.tick_params(labelsize=30)

In [ ]:
set_plt_params()
results = []

for X, name, lstyle in [(glove_vectors, 'Glove', 'dotted'),
                (word2vec_vectors, 'Word2Vec', 'dashed'),
                (wang2vec_vectors, 'Wang2Vec', 'dashdot'),
                (w2vf_vectors, 'Word2Vec-f',  (0, (20,10))),
                (adagram_vectors, 'Adagram', (0,(10,10))),
                (ft_vectors, 'FastText','solid'),
                (bow_vectors, 'Bag of Words',(0,(30,10))),
                (swivel_vectors, 'Swivel',(0,(40,10))),
                ]:
    #estimator = SVC(C = 0.01, kernel = 'linear')
    estimator = KNeighborsClassifier(n_neighbors = 3)
    cv = ShuffleSplit(n_splits=20, test_size=0.1, random_state=0)
    train_sizes=np.linspace(0.1, 0.9, 5)
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, Y, cv=cv, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    results.append({'model' : name, 'score' : train_scores_mean[-1]})
    plt.plot(train_sizes, train_scores_mean, label=name, linewidth=5, linestyle=lstyle)

plt.legend(loc="upper left", bbox_to_anchor=(1,1), prop={'size':30})
plt.savefig('classifiers.png', bbox_inches='tight')
plt.show()

In [ ]:
results